In [1]:
import pandas as pd
import numpy as np
import itertools
import os
from IPython.display import display, HTML


from Event_detection_functions import *


In [3]:
# Decleration of dataframes 
Event_df = pd.DataFrame(columns = ['timestamp','Phase','minuten_index',
                                   'Ladevorgang', 'Status', 'Ladeleistung',
                                   'minuten_index_Abschaltung','timestamp_abschalt'])
pseudocode_df = pd.DataFrame([])


In [5]:
#test loop, delete later
for minute_day in range (2,690):

    # Insert this in the update routine after the power_today_minute.csv export
    # The following variables have to be set to the corresponding dataframe in the environment (export to power_today_minute_x.csv)
    P_Phase_1 = pd.read_csv('power_today_minute_1.csv', names = ['timestamp','P'])
    P_Phase_1 = P_Phase_1[:minute_day]
    P_Phase_2 = pd.read_csv('power_today_minute_2.csv', names = ['timestamp','P'])
    P_Phase_2 = P_Phase_2[:minute_day]
    P_Phase_3 = pd.read_csv('power_today_minute_3.csv', names = ['timestamp','P'])
    P_Phase_3 = P_Phase_3[:minute_day]

    dict = {1:P_Phase_1,2:P_Phase_2,3:P_Phase_3}

    #Detection of switch-on and switch-off events
    for Phase in range (1,4):
        
        pseudocode_df = dict[Phase]
        pseudocode_df['P_delta'] = dict[Phase].P.diff()
        Event_df = detect_switch_event(pseudocode_df, Phase, Event_df)

        


C:\Users\Admin\Event_detection_functions.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  only_load_event_df['deviation_P'] = only_load_event_df['Ladeleistung'].                                     apply(lambda ladeleistung_auto: abs(ladeleistung_auto-P_residual)/ladeleistung_auto)


In [6]:
Event_df

,timestamp,Phase,minuten_index,Ladevorgang,Status,Ladeleistung,minuten_index_Abschaltung,timestamp_abschalt
0,1.507176e+09,3,357,aus,checked,3451,687,1.5072e+09
1,1.507194e+09,1,666,ein,checked,3575,0,0


In [7]:
ladevorgang_df = combine_charging_events(Event_df)
ladevorgang_df

C:\Users\Admin\Event_detection_functions.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  same_events['deviation_average'] = same_events['Ladeleistung'].                                     apply(lambda ladeleistung_auto: ladeleistung_auto/(same_events['Ladeleistung'].mean()))


,timestamp_start,Phasen,Durchschnittliche Ladeleistung,Ladevorgang_ende
0,1.507176e+09,[3],[3: 3451.0],1.507196e+09
1,1.507194e+09,[1],[1: 3575.0],0.000000e+00


In [9]:
#timestamp has to be replaced by variable used in py script
timestamp = (pd.read_csv('power_today_minute_1.csv', names = ['timestamp','P'])).iloc[700]['timestamp']

charging_df=cars_charging(timestamp,ladevorgang_df)
charging_df

C:\Users\Admin\Event_detection_functions.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  current_cars['length_phase'] = current_cars['Phasen'].apply(lambda x: len(x))
C:\Users\Admin\Event_detection_functions.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  current_cars['Phase_1'] = current_cars['Phasen'].apply(lambda x: 1 in x)
C:\Users\Admin\Event_detection_functions.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

,timestamp,cars_total,one_phase,two_phases,three_phases,cars_phase_1,cars_phase_2,cars_phase_3
0,1.507196e+09,1.0,1.0,0.0,0.0,1.0,0.0,0.0


In [10]:
#export
charging_df.to_csv('charging_df.csv', )
